In [68]:
%run ./a/import.ipynb
%run ./a/aroon.ipynb
%run ./a/bollinger_bands.ipynb
%run ./a/CHOP.ipynb
%run ./a/DMI.ipynb
%run ./a/EMA_MACD.ipynb
%run ./a/MFI.ipynb
%run ./a/OBV.ipynb
%run ./a/OHLC.ipynb
%run ./a/pivot_pt_hl.ipynb
%run ./a/pivot_pt_rs.ipynb
%run ./a/ROC.ipynb
%run ./a/RSI.ipynb
%run ./a/HLC_stochastic.ipynb

In [69]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [70]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.core.display import display, HTML
%matplotlib notebook

In [71]:
company = 'MSFT'
API = 'yahoo'
start = '2016-01-01'
end = '2019-12-31'

In [74]:
dfo = data.DataReader(company, API, start, end)

In [83]:
def plot(OHLC_day=10, Aroon=False, Aroon_period=25, Bollinger_Bands=False, Smoothing_period=20, SD=2, CHOP=True, CHOP_period=14, DMI=False, DMI_period=14, 
         MACD=False, EMA_period=12, EMA_fitting=2, MACD_period=26, Signal_period=9, MFI=True, MFI_period=14, OBV=False, 
         Pivot_HL=False, HL_period=21, Pivot_RS=False, RS_period=21, ROC=False, ROC_period=25, RSI=False, RSI_period=14, HLC_Stochastic=False, HLC_period=14, HLC_period_Avg=3):
    global dfo
    
    number = 1

    if Aroon:
        number+=1
    if Bollinger_Bands:
        number+=3
    if CHOP:
        number+=1
    if DMI:
        number+=1
    if MACD:
        number+=2
    if MFI:
        number+=1
    if OBV:
        number+=1
    if ROC:
        number+=1
    if Pivot_HL:
        number+=1
    if Pivot_RS:
        number+=1
    if RSI:
        number+=1
    if HLC_Stochastic:
        number+=2

    fig, axes = plt.subplots(number, 1, figsize=(12.2, 4.5 * number), sharex=True)
    
    try:
        df1 = ohlc(dfo, OHLC_day)
        
        candlestick_ohlc(axes[0], df1.values, width=OHLC_day/2, colorup='g')

        axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

        number-=1
    except:
        df1 = ohlc(dfo, OHLC_day)
        
        candlestick_ohlc(axes, df1.values, width=OHLC_day/2, colorup='g')

        axes.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

        number-=1
    
    if Aroon:
        df1 = aroon(dfo, Aroon_period)

        df1[['aroon_up','aroon_down']].plot(ax=axes[number])
        axes[number].set_title('Aroon')
        axes[number].axhline(50, linestyle='--', alpha = 0.5, color = 'red', label='Breakout Line')
        axes[number].legend()
        
        number-=1
        
    if Bollinger_Bands:
        df1 = bollinger(dfo, Smoothing_period, SD)

        axes[number-2].set_title('Bollinger Bands')
        axes[number-2].plot(df1.index, df1['SMA'], color='black', alpha=0.5)
        axes[number-2].plot(df1.index, df1['BOLU'], color='orange', alpha=0.8)
        axes[number-2].plot(df1.index, df1['BOLD'], color='green', alpha=0.5)
        axes[number-2].plot(df1.index, df1['Close'], color='red')

        axes[number-1].plot(df1.index, df1['%b'], label='%b')

        axes[number].plot(df1.index, df1['band_width'], label='band_width')
        
        axes[number-1].legend()
        axes[number].legend()
    
        number-=3
        
    if CHOP:
        df1 = chop(dfo, CHOP_period)
        
        axes[number].set_title('CHOP')
        axes[number].plot(df1)
        axes[number].axhline(61.8, linestyle='--', alpha = 0.5, color = 'red', label='Consolidation')
        axes[number].axhline(38.2, linestyle='--', alpha = 0.5, color = 'green', label='Trend Start')
        axes[number].legend(loc='lower right')
        
        number-=1
        
    if DMI:
        df1 = dmi(dfo, DMI_period)

        df1['smooth_+DMI'].plot(ax=axes[number], label='+DMI', color = 'orange'); axes[number]
        df1['smooth_-DMI'].plot(ax=axes[number], label='-DMI', color = 'purple'); axes[number]
        df1['ADX'].plot(ax=axes[number], label='ADX', color = 'red'); axes[number].axhline(25, linestyle='--', alpha = 0.5, color = 'blue', label='Trend Start')
        axes[number].legend()
        
        number-=1
        
    if MACD:
        df1 = macd(dfo, EMA_period, EMA_fitting, MACD_period, Signal_period)

        axes[number-1].plot(df.index, df['Close'], color='blue', label='Close')
        axes[number-1].plot(df1.index, df1.filter(regex='ema$', axis=1).iloc[:,0], 
                 label=[col for col in df1.columns if 'ema' in col][0])
        axes[number-1].plot(df1.index, df1.filter(regex='ema$', axis=1).iloc[:,1], 
                 label=[col for col in df1.columns if 'ema' in col][1])
        axes[number-1].legend()

        axes[number].set_title('MACD')
        axes[number].plot(df1.index, df1['MACD'], label=[col for col in df1.columns if 'ema' in col][0] + '-' + [col for col in df1.columns if 'ema' in col][1])
        axes[number].plot(df1.index, df1['signal'], label='avg' + [col for col in df1.columns if 'ema' in col][0] + '-' + [col for col in df1.columns if 'ema' in col][1])
        axes[number].plot(df1.index, df1['MACD_up'], color='g')
        axes[number].plot(df1.index, df1['MACD_down'], color='r')
        axes[number].fill_between(df1.index, df1['MACD_up'], 0, alpha=0.3, color='green')
        axes[number].fill_between(df1.index, df1['MACD_down'], 0, alpha=0.3, color='red')
        axes[number].legend()

        number-=2
        
    if MFI:
        df1 = mfi(dfo, MFI_period)
            
        axes[number].set_title('MFI')
        axes[number].plot(df1)
        axes[number].axhline(10, linestyle='--', alpha = 0.5, color = 'green', label='Oversold')
        axes[number].axhline(90, linestyle='--', alpha = 0.5, color = 'red', label='Overbought')
        axes[number].legend()
        
        number-=1
        
    if OBV:
        if not 'OBV' in dfo:
            df1 = obv(dfo)
            dfo = pd.concat([dfo, df1], axis=1)

        axes[number].set_title('OBV')
        axes[number].plot(df.index, df['OBV'])

        number-=1
        
    if ROC:
        df1 = roc(dfo, ROC_period)
        
        axes[number].set_title('ROC')
        axes[number].plot(df1)
        
        number-=1
        
    if Pivot_HL:
        df1 = pivot_hl(dfo, HL_period)
        
        axes[number].plot(dfo.index, dfo['Close'])
        for i in range(len(df1.index)):
            if df1['h_index'][i]:
                axes[number].plot(dfo.index[i], dfo['High'][i], marker=7, color='red')
            elif df1['l_index'][i]:
                axes[number].plot(dfo.index[i], dfo['Low'][i], marker=6, color='green')
                
        axes[number].set_title('Period High and Low')

        number-=1
        
    if Pivot_RS:
        df1 = pivot_rs(dfo, RS_period)

        for i in range( len(df1.index) - 1 ):
            axes[number].plot_date([df1.index[i], df1.index[i+1]], 
                         [df1['pivot_point'][i], df1['pivot_point'][i]], linestyle="-", lw=1, marker=",", color='blue', alpha = 0.5)
            axes[number].plot_date([df1.index[i], df1.index[i+1]], 
                         [df1['s1'][i], df1['s1'][i]], linestyle="--", lw=1, marker=",", color='red', alpha = 0.5)
            axes[number].plot_date([df1.index[i], df1.index[i+1]], 
                          [df1['s2'][i], df1['s2'][i]], linestyle="-.", lw=1, marker=",", color='orange', alpha = 0.5)
            axes[number].plot_date([df1.index[i], df1.index[i+1]], 
                         [df1['r1'][i], df1['r1'][i]], linestyle="--", lw=1, marker=",", color='red', alpha = 0.5)
            axes[number].plot_date([df1.index[i], df1.index[i+1]], 
                         [df1['r2'][i], df1['r2'][i]], linestyle="-.", lw=1, marker=",", color='orange', alpha = 0.5)
            axes[number].plot_date([df1.index[i], df1.index[i+1]], 
                         [df1['s3'][i], df1['s3'][i]], linestyle=":", lw=1, marker=",", color='yellow', alpha = 0.8)
            axes[number].plot_date([df1.index[i], df1.index[i+1]], 
                         [df1['r3'][i], df1['r3'][i]], linestyle=":", lw=1, marker=",", color='yellow', alpha = 0.8)
            axes[number].plot(dfo.index, dfo['Close'], color='red')

        number-=1
        
    if RSI:
        df1 = rsi(dfo, RSI_period)

        axes[number].set_title('RSI')
        axes[number].plot(df1)
        axes[number].axhline(0, linestyle='--', alpha = 0.5, color = 'red')
        axes[number].axhline(10, linestyle='--', alpha = 0.5, color = 'orange')
        axes[number].axhline(20, linestyle='--', alpha = 0.5, color = 'yellow')
        axes[number].axhline(30, linestyle='--', alpha = 0.5, color = 'grey')
        axes[number].axhline(70, linestyle='--', alpha = 0.5, color = 'grey')
        axes[number].axhline(80, linestyle='--', alpha = 0.5, color = 'yellow')
        axes[number].axhline(90, linestyle='--', alpha = 0.5, color = 'orange')
        axes[number].axhline(100, linestyle='--', alpha = 0.5, color = 'red')
    
        number-=1
        
    if HLC_Stochastic:
        df1 = hlc_stochastic(dfo, HLC_period, HLC_period_Avg)

        df1[['%K','%D']].plot(ax=axes[number-1]); axes[number-1].set_title('Close Oscillator')
        axes[number-1].axhline(20, linestyle='--', alpha = 0.5, color = 'green')
        axes[number-1].axhline(80, linestyle='--', alpha = 0.5, color = 'red')
        
        (df1['%K'] - df1['%D']).plot(ax=axes[number]); axes[number].set_title('%K-%D')
        axes[number].axhline(0, linestyle='--', alpha = 0.5, color = 'green')
        
        number-=1

style = {'description_width': 'initial'}
interact_manual(plot, 
                a=widgets.Checkbox(Aroon=False, Bollinger_Bands=False, CHOP=True, DMI=False, MACD=False, MFI=True, OBV=False, 
                                   Pivot_HL=False, Pivot_RS=False,ROC=False, RSI=False, HL_Stochastic=False), 
                OHLC_day=widgets.IntSlider(min=5, max=30, step=1,value=10, style=style),
                Aroon_period=widgets.IntSlider(min=5, max=60, step=1,value=25, style=style), 
                Smoothing_period=widgets.IntSlider(min=5, max=60, step=1,value=20, style=style), 
                SD=widgets.FloatSlider(min=1, max=5, step=0.1,value=2, style=style), 
                CHOP_period=widgets.IntSlider(min=5, max=60, step=1,value=14, style=style), 
                DMI_period=widgets.IntSlider(min=5, max=60, step=1,value=14, style=style), 
                EMA_period=widgets.IntSlider(min=2, max=100, step=1,value=12, style=style), 
                EMA_fitting=widgets.FloatSlider(min=1, max=5, step=0.1,value=2, style=style), 
                MACD_period=widgets.IntSlider(min=2, max=200, step=1,value=26, style=style), 
                Signal_period=widgets.IntSlider(min=2, max=100, step=1,value=9, style=style), 
                MFI_period=widgets.IntSlider(min=5, max=60, step=1,value=14, style=style), 
                ROC_period=widgets.IntSlider(min=5, max=200, step=1,value=25, style=style), 
                HL_period=widgets.IntSlider(min=5, max=61, step=2,value=21, style=style), 
                RS_period=widgets.IntSlider(min=5, max=60, step=1,value=26, style=style), 
                RSI_period=widgets.IntSlider(min=5, max=60, step=1,value=14, style=style), 
                HLC_period=widgets.IntSlider(min=5, max=60, step=1,value=14, style=style), 
                HLC_period_Avg=widgets.IntSlider(min=1, max=20, step=1,value=3, style=style))

interactive(children=(IntSlider(value=10, description='OHLC_day', max=30, min=5, style=SliderStyle(description…

<function __main__.plot(OHLC_day=10, Aroon=False, Aroon_period=25, Bollinger_Bands=False, Smoothing_period=20, SD=2, CHOP=True, CHOP_period=14, DMI=False, DMI_period=14, MACD=False, EMA_period=12, EMA_fitting=2, MACD_period=26, Signal_period=9, MFI=True, MFI_period=14, OBV=False, Pivot_HL=False, HL_period=21, Pivot_RS=False, RS_period=21, ROC=False, ROC_period=25, RSI=False, RSI_period=14, HLC_Stochastic=False, HLC_period=14, HLC_period_Avg=3)>